# Transform Addresses Data
1. Create one record for each customer with 2 sets of address column, 1 for shipping and 1 for billing address
2. Write Transformed Data to Silver Schema

In [0]:
select *
from gizmobox.bronze.v_addresses;

### 1. Create one record for each customer with 2 sets of address column, 1 for shipping and 1 for billing address

In [0]:
select customer_id, 
max(case
  when address_type = 'shipping' then address_line_1
end) as shipping_address,
max(case
  when address_type = 'shipping' then city
end) as shipping_city,
max(case
  when address_type = 'shipping' then state
end) as shipping_state,
max(case
  when address_type = 'shipping' then postcode
end) as shipping_postcode,
max(case
  when address_type = 'billing' then address_line_1
end) as billing_address,
max(case
  when address_type = 'billing' then city
end) as billing_city,
max(case
  when address_type = 'billing' then state
end) as billing_state,
max(case
  when address_type = 'billing' then postcode
end) as billing_postcode
from gizmobox.bronze.v_addresses
group by customer_id;

In [0]:
-- Using PIVOT()
select * from(
              select customer_id,
              address_type,
              address_line_1,
              city,
              state,
              postcode
              from gizmobox.bronze.v_addresses)
pivot (
  max(address_line_1) as address_line_1,
  max(city) as city,
  max(state) as state,
  max(postcode) as postcode
  for address_type in ('shipping', 'billing')
);

### 2. Write Transformed Data to Silver Schema

In [0]:
create table gizmobox.silver.addresses
as
select * from(
              select customer_id,
              address_type,
              address_line_1,
              city,
              state,
              postcode
              from gizmobox.bronze.v_addresses)
pivot (
        max(address_line_1) as address_line_1,
        max(city) as city,
        max(state) as state,
        max(postcode) as postcode
        for address_type in ('shipping', 'billing')
      );

In [0]:
select * from gizmobox.silver.addresses;